# Rf 2 0 3_Ranges
Addition and convolution: fitting and plotting in sub ranges



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:43 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooPolynomial.h"
#include "RooAddPdf.h"
#include "RooFitResult.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup model
 ---------------------

Construct observables x

In [3]:
RooRealVar x("x", "x", -10, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct gaussx(x,mx,1)

In [4]:
RooRealVar mx("mx", "mx", 0, -10, 10);
RooGaussian gx("gx", "gx", x, mx, RooConst(1));

Construct px = 1 (flat in x)

In [5]:
RooPolynomial px("px", "px", x);

Construct model = f*gx + (1-f)px

In [6]:
RooRealVar f("f", "f", 0., 1.);
RooAddPdf model("model", "model", RooArgList(gx, px), f);

Generated 10000 events in (x,y) from p.d.f. model

In [7]:
RooDataSet *modelData = model.generate(x, 10000);

Fit full range
 ---------------------------

Fit p.d.f to all data

In [8]:
RooFitResult *r_full = model.fitTo(*modelData, Save(kTRUE));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (px)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gx)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 mx           0.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 

Fit partial range
 ----------------------------------

Define "signal" range in x as [-3,3]

In [9]:
x.setRange("signal", -3, 3);

[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'signal' created with bounds [-3,3]


Fit p.d.f only to data in "signal" range

In [10]:
RooFitResult *r_sig = model.fitTo(*modelData, Save(kTRUE), Range("signal"));

[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_model_modelData) constructing test statistic for sub-range named signal
[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'NormalizationRangeForsignal' created with bounds [-10,10]
[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'fit_nll_model_modelData' created with bounds [-3,3]
[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_model_modelData) fixing interpretation of coefficients of any RooAddPdf to full domain of observables 
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (px)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gx)
 **********
 **   10 **SET PRINT           1
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMI

Plot / print results
 ---------------------------------------

Make plot frame in x and add data and fitted model

In [11]:
RooPlot *frame = x.frame(Title("Fitting a sub range"));
modelData->plotOn(frame);
model.plotOn(frame, Range("Full"), LineStyle(kDashed), LineColor(kRed)); // Add shape in full ranged dashed
model.plotOn(frame);                                                     // By default only fitted range is shown

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f was fitted in range and no explicit norm range was specified, using fit range as default
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) only plotting range 'Full'
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f. curve is normalized using explicit choice of ranges 'fit_nll_model_modelData'
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f was fitted in range and no explicit plot,norm range was specified, using fit range as default
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) only plotting range 'fit_nll_model_modelData'
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f. curve is normalized using explicit choice of ranges 'fit_nll_model_modelData'


Print fit results

In [12]:
cout << "result of fit on all data " << endl;
r_full->Print();
cout << "result of fit in in signal region (note increased error on signal fraction)" << endl;
r_sig->Print();

result of fit on all data 

  RooFitResult: minimized FCN value: 25939.4, estimated distance to minimum: 7.58116e-06
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                     f    5.0441e-01 +/-  6.32e-03
                    mx   -2.1605e-02 +/-  1.77e-02

result of fit in in signal region (note increased error on signal fraction)

  RooFitResult: minimized FCN value: 10339.5, estimated distance to minimum: 3.38988e-07
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                     f    4.9014e-01 +/-  1.62e-02
                    mx   -2.1701e-02 +/-  1.79e-02



Draw frame on canvas

In [13]:
new TCanvas("rf203_ranges", "rf203_ranges", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();

return;

Draw all canvases 

In [14]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()